In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

/home/ospanoff/.pyenv/versions/3.5.1/envs/python3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sys import stdout
import time

In [3]:
data_train = pd.read_csv("data/train.csv")
data_test = pd.read_csv("data/test.csv")

In [4]:
data_all = data_train.append(data_test)
del data_train
del data_test

In [7]:
X = data_all.drop([col for col in data_all.columns if 'cat' in col and col not in ['cat' + str(i) for i in range(89, 117)]], axis=1)

In [8]:
for i in range(6):
    cols = ['cat' + str(12 * i + j) for j in range(1, 13)]
    X['cat_couples_' + str(i)] = data_all[cols].apply(lambda x: x.sum(), axis=1)

In [9]:
X['cat_triples'] = data_all[['cat' + str(i) for i in range(73, 77)]].apply(lambda x: x.sum(), axis=1)

In [10]:
X['cat_quadro'] = data_all[['cat' + str(i) for i in range(77, 89)]].apply(lambda x: x.sum(), axis=1)

In [5]:
X = data_all.copy()
del data_all

In [6]:
def make_cardin_code(cat_feat):
    uniq = set(cat_feat)
    ret = dict()
    for i, f in enumerate(uniq):
        stdout.write("\r%0.2f%%" % (100 * (i + 1) / len(uniq)))
        ret[f] = (cat_feat == f).mean()

    return ret

In [7]:
for col in X.columns:
    if 'cat' in col:
        print(col)
        uniq = make_cardin_code(X[col].values)
        X[col + '_coded'] = X[col].apply(lambda x: uniq[x])
        print('\n')

cat1
100.00%

cat10
100.00%

cat100
100.00%

cat101
100.00%

cat102
100.00%

cat103
100.00%

cat104
100.00%

cat105
100.00%

cat106
100.00%

cat107
100.00%

cat108
100.00%

cat109
100.00%

cat11
100.00%

cat110
100.00%

cat111
100.00%

cat112
100.00%

cat113
100.00%

cat114
100.00%

cat115
100.00%

cat116
100.00%

cat12
100.00%

cat13
100.00%

cat14
100.00%

cat15
100.00%

cat16
100.00%

cat17
100.00%

cat18
100.00%

cat19
100.00%

cat2
100.00%

cat20
100.00%

cat21
100.00%

cat22
100.00%

cat23
100.00%

cat24
100.00%

cat25
100.00%

cat26
100.00%

cat27
100.00%

cat28
100.00%

cat29
100.00%

cat3
100.00%

cat30
100.00%

cat31
100.00%

cat32
100.00%

cat33
100.00%

cat34
100.00%

cat35
100.00%

cat36
100.00%

cat37
100.00%

cat38
100.00%

cat39
100.00%

cat4
100.00%

cat40
100.00%

cat41
100.00%

cat42
100.00%

cat43
100.00%

cat44
100.00%

cat45
100.00%

cat46
100.00%

cat47
100.00%

cat48
100.00%

cat49
100.00%

cat5
100.00%

cat50
100.00%

cat51
100.00%

cat52
100.00%

cat53
100.00%

In [8]:
X.drop([col for col in X.columns if 'cat' in col and '_coded' not in col], axis=1, inplace=True)

In [10]:
from sklearn.metrics import mean_absolute_error

In [11]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

In [13]:
from sklearn.model_selection import KFold

In [18]:
kf = KFold(n_splits=5)

In [28]:
train = X[X['loss'].notnull()]
test = X[X['loss'].isnull()]

shift = 200
y = np.log(train['loss'] + shift)
X_train = train.drop(['loss', 'id'], 1)
X_test = test.drop(['loss', 'id'], 1)

RANDOM_STATE = 2016
params = {
    'min_child_weight': 1,
    'eta': 0.01,
    'colsample_bytree': 0.5,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 1,
    'gamma': 1,
    'silent': 1,
    'verbose_eval': True,
    'seed': RANDOM_STATE
}

prediction = np.zeros(X_test.shape[0])

for tr_ind, te_ind in kf.split(X_train):

    xgtrain = xgb.DMatrix(X_train.values[tr_ind], label=y[tr_ind])
    xgtest = xgb.DMatrix(X_test.values)

    model = xgb.train(params, xgtrain, int(2012 / 0.9), feval=evalerror)

    prediction += np.exp(model.predict(xgtest)) - shift

submission = pd.DataFrame()
submission['loss'] = prediction / 5
submission['id'] = test['id']
submission.to_csv('sub_my.csv', index=False)